In [1]:
import pandas as pd
import collections as c
from collections import Counter
import re
import csv

In [2]:
df = pd.read_csv('Food_Inspections.csv')

In [3]:
df.head(5)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2484567,ROYALTY,ROYALTY,1306130.0,Restaurant,Risk 1 (High),3810 W 63RD ST,CHICAGO,IL,60629.0,02/05/2021,Non-Inspection,No Entry,NaN,41.778837,-87.718361,"(-87.71836138998039, 41.778836516734856)"
1,2473041,ROSATI'S GRANT PARK,ROSATI'S,2762683.0,Restaurant,Risk 1 (High),23 E ADAMS ST,CHICAGO,IL,60603.0,01/22/2021,License,Pass,NaN,41.879391,-87.626848,"(-87.62684825563626, 41.879391313239694)"
2,2463667,M & M FOOD,M & M FOOD,1193207.0,NaN,Risk 3 (Low),7073 N WESTERN AVE,CHICAGO,IL,60645.0,12/11/2020,Canvass,Out of Business,NaN,42.010769,-87.690067,"(-87.69006746560828, 42.01076851593425)"
3,2457076,SUBWAY,SUBWAY,2411733.0,Restaurant,Risk 1 (High),4406 S PULASKI RD,CHICAGO,IL,60632.0,11/27/2020,Canvass,Pass,NaN,41.813101,-87.723928,"(-87.72392832288558, 41.81310105112748)"
4,2456274,TRATTORIA ULTIMO,TRATTORIA ULTIMO,2069755.0,Restaurant,Risk 1 (High),1953 W CHICAGO AVE,CHICAGO,IL,60622.0,11/05/2020,Canvass,Out of Business,NaN,41.895808,-87.676752,"(-87.67675213462381, 41.89580774819584)"


In [4]:
Results = df['Results']
Results.unique()

In [5]:
Results.unique()

array(['No Entry', 'Pass', 'Out of Business', 'Not Ready', 'Fail',
       'Pass w/ Conditions', 'Business Not Located'], dtype=object)

In [6]:
Fail = df["Results"] == "Fail"
Failed = df.loc[Fail]


In [7]:
Failed.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
27,2373427,THE BUNKER,THE BUNKER,2718517.0,Restaurant,Risk 3 (Low),1515 W MONROE ST,CHICAGO,IL,60607.0,06/09/2020,License,Fail,NaN,41.880086,-87.665054,"(-87.66505441566734, 41.88008566659281)"
35,2370354,SLIM'S,SLIM'S,2622236.0,Restaurant,Risk 1 (High),2020 W MONTROSE AVE,CHICAGO,IL,60618.0,05/08/2020,Complaint,Fail,"1. PERSON IN CHARGE PRESENT, DEMONSTRATES KNOW...",41.961556,-87.679862,"(-87.67986227331792, 41.96155579127668)"
41,2369855,Chicago Prep Master Tenant LLC,Chicago Prep Master Tenant,2723671.0,Shared Kitchen,Risk 1 (High),5801 W Dickens AVE,CHICAGO,IL,60639.0,04/28/2020,License,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.917869,-87.770697,"(-87.77069733431132, 41.91786924215764)"
48,2369400,DSD DELI,DSD DELI,2626178.0,DELI/GROCERY,Risk 1 (High),5205 N MILWAUKEE,CHICAGO,IL,60630.0,04/17/2020,License,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",41.974849,-87.766829,"(-87.76682905043768, 41.97484853157327)"
49,2369394,AMSTAR,AMSTAR,2152845.0,Grocery Store,Risk 2 (Medium),8950-58 S ASHLAND AVE,CHICAGO,IL,60620.0,04/17/2020,Canvass Re-Inspection,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.730593,-87.663058,"(-87.66305762988213, 41.730593381254266)"


In [8]:
Failed = Failed.dropna(subset=['Violations'])

In [9]:
Failed['Violations'].isnull()

35        False
41        False
48        False
49        False
59        False
60        False
64        False
148       False
153       False
160       False
164       False
165       False
166       False
211       False
217       False
220       False
242       False
248       False
259       False
269       False
299       False
310       False
312       False
320       False
321       False
332       False
334       False
338       False
342       False
345       False
          ...  
218067    False
218071    False
218077    False
218078    False
218079    False
218081    False
218083    False
218087    False
218091    False
218101    False
218106    False
218111    False
218123    False
218126    False
218128    False
218134    False
218148    False
218149    False
218150    False
218154    False
218157    False
218161    False
218162    False
218169    False
218170    False
218172    False
218176    False
218180    False
218187    False
218191    False
Name: Violations, Length

In [10]:
violation_master_list = []
f = lambda x: x['Violations'].split("|")
violation_list = Failed.apply(f, axis=1)

In [11]:
flat_list = [item for sublist in violation_list for item in sublist]

In [12]:
x = []
pattern = r"[0-9]+\.(.*?)(?=Comments:)"

for i in flat_list:
     result = re.findall(pattern,str(i)) #perform regex & identify/extract the violation code only
     x.append(result)#append the violation code into violation_master_list


In [13]:
x1 = tuple(tuple(p) for p in x)

In [15]:
Counter(x1)

Counter({(' PERSON IN CHARGE PRESENT, DEMONSTRATES KNOWLEDGE, AND PERFORMS DUTIES - ',): 657,
         (' CITY OF CHICAGO FOOD SERVICE SANITATION CERTIFICATE - ',): 1330,
         (' MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - ',): 3971,
         (' PROCEDURES FOR RESPONDING TO VOMITING AND DIARRHEAL EVENTS - ',): 4409,
         (' NO BARE HAND CONTACT WITH RTE FOOD OR A PRE-APPROVED ALTERNATIVE PROCEDURE PROPERLY ALLOWED - ',): 107,
         (' ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE - ',): 4727,
         (' FOOD SEPARATED AND PROTECTED - ',): 130,
         (' PROPER HOT HOLDING TEMPERATURES - ',): 213,
         (' PROPER DATE MARKING AND DISPOSITION - ',): 1567,
         (' THERMOMETERS PROVIDED & ACCURATE - ',): 1860,
         (' FOOD PROPERLY LABELED; ORIGINAL CONTAINER - ',): 1659,
         (' INSECTS, RODENTS, & ANIMALS NOT PRESENT - ',): 4832,
         (' PERSONAL CLEANLINESS - ',): 519,
         (' WIPING C

In [14]:
Counter(x1).most_common(10) 

[((' FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - ',),
  19343),
 ((' WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - ',),
  18225),
 ((' FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSILS CLEAN, FREE OF ABRASIVE DETERGENTS - ',),
  16425),
 ((' NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - ',),
  16420),
 ((' VENTILATION: ROOMS AND EQUIPMENT VENTED AS REQUIRED: PLUMBING: INSTALLED AND MAINTAINED - ',),
  15508),
 ((' FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - ',),
  14995),
 ((' PREMISES MAINTAINED FREE OF LITTER, UNNECESSARY ARTICLES, CLEANING  EQUIPMENT PROPERLY STORED - ',),
  10652),
 ((' PHYSICAL FACILITIES INSTALLED, MAINTAINED & CLEAN - ',), 9310),
 ((' LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF LIGHT PROVIDED, FIXTURES SHIELDE